In [ ]:
!pip install PyMuPDF pdfminer.six sentence-transformers faiss-cpu elasticsearch langchain gpt4all

In [ ]:
import fitz
import re

In [ ]:
doc = fitz.open("/content/drive/MyDrive/1706.03762v7.pdf")  # Replace with the path to your PDF

In [ ]:
text = ""
for page_num in range(10):
  page = doc.load_page(page_num)
  text += page.get_text("text")

In [ ]:
text = re.sub(r'\n\d+\n', '\n', text) # remove page number
text = re.sub(r'\n+', '\n', text) # remove multiple newlines
text = re.sub(r'-\n', '', text) # remove hyphenated line breaks
text = re.sub(r'\[\s*(\d+,\s*)*\d+\s*\]', '', text) #references
text = re.sub(r'\.\n', '. ## ', text) # for splitting
text = re.sub(r'\n', ' ', text) # remove line break
text = re.sub(r'  ', ' ', text) # remove double space

In [ ]:
chunks = text.split("##")
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

In [ ]:
delete = ['Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗† University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗‡ illia.polosukhin@gmail.com',
 'arXiv:1706.03762v7 [cs.CL] 2 Aug 2023',
 'Scaled Dot-Product Attention Multi-Head Attention Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.',
 '4To illustrate why the dot products get large, assume that the components of q and k are independent random variables with mean 0 and variance 1. Then their dot product, q · k = Pdk i=1 qiki, has mean 0 and variance dk.',
 '5We used values of 2.8, 3.7, 6.0 and 9.5 TFLOPS for K80, K40, M40 and P100, respectively.',
 'Model BLEU Training Cost (FLOPs) EN-DE EN-FR EN-DE EN-FR ByteNet 23.75 Deep-Att + PosUnk 39.2 1.0 · 1020 GNMT + RL 24.6 39.92 2.3 · 1019 1.4 · 1020 ConvS2S 25.16 40.46 9.6 · 1018 1.5 · 1020 MoE 26.03 40.56 2.0 · 1019 1.2 · 1020 Deep-Att + PosUnk Ensemble 40.4 8.0 · 1020 GNMT + RL Ensemble 26.30 41.16 1.8 · 1020 1.1 · 1021 ConvS2S Ensemble 26.36 41.29 7.7 · 1019 1.2 · 1021 Transformer (base model) 27.3 38.1 3.3 · 1018 Transformer (big) 28.4 41.8 2.3 · 1019 ',
 'Layer Type Complexity per Layer Sequential Maximum Path Length Operations Self-Attention O(n2 · d) O(1) O(1) Recurrent O(n · d2) O(n) O(n) Convolutional O(k · n · d2) O(1) O(logk(n)) Self-Attention (restricted) O(r · n · d) O(1) O(n/r) ',
 'Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23 of WSJ) Parser Training WSJ 23 F1 Vinyals & Kaiser el al. (2014) WSJ only, discriminative 88.3 Petrov et al. (2006) WSJ only, discriminative 90.4 Zhu et al. (2013) WSJ only, discriminative 90.4 Dyer et al. (2016) WSJ only, discriminative 91.7 Transformer (4 layers) WSJ only, discriminative 91.3 Zhu et al. (2013) semi-supervised 91.3 Huang & Harper (2009) semi-supervised 91.3 McClosky et al. (2006) semi-supervised 92.1 Vinyals & Kaiser el al. (2014) semi-supervised 92.1 Transformer (4 layers) semi-supervised 92.7 Luong et al. (2015) multi-task 93.0 Dyer et al. (2016) generative 93.3 ']

In [ ]:
new_chunks = []
for chunk in chunks:
  for substring in delete:
    while substring in chunk:
      chunk = chunk.replace(substring, '', 1)
      print('deleted: ', substring)
  new_chunks.append(chunk)
chunks = new_chunks

In [ ]:
index_to_remove = [0,2,3,4,5,17,18,48,73,74,75,76,77]

In [ ]:
new_chunks = []
for i in range(len(chunks)):
  if i not in index_to_remove:
    new_chunks.append(chunks[i])

In [ ]:
join = [4, 10, 14, 19, 21, 28, 36, 45, 59]

In [ ]:
strings = new_chunks
for i in range(len(join)):
  joined_string = strings[join[i]-i] +" "+ strings[join[i]-i+1]
  strings = strings[:join[i]-i] + [joined_string] + strings[join[i]-i+2:]
  print(joined_string)

In [ ]:
new_chunks = strings

In [ ]:
for i, chunk in enumerate(new_chunks):
  print(f"{i} ", len(chunk), " ", chunk,"\n")

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

embeddings = embedding_model.encode(new_chunks, convert_to_tensor=True)

embeddings = embeddings.cpu()

torch.save(embeddings, "embeddings.pt")

In [ ]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings.numpy())

query = "What is the main purpose of the paper?"
query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().numpy()

k = 5
distances, indices = index.search(query_embedding.reshape(1, -1), k)

for idx in indices[0]:
  print(f"Chunk {idx}: {new_chunks[idx]}")

In [ ]:
def relevant_chunks(query, k=5):
  query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().numpy()
  distances, indices = index.search(query_embedding.reshape(1, -1), k)
  relevant_chunks = [new_chunks[idx] for idx in indices[0]]
  return relevant_chunks

query = "What is an encoder?"
relevant_chunk = relevant_chunks(query, k=5)

context = " ".join(relevant_chunk)
print("Context for generation:\n", context)

In [ ]:
from gpt4all import GPT4All

model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf", allow_download=True)

def generate_response(context, query):
  prompt = f"Answer this question: {query}\n\nBased on this context: {context}"
  response = model.generate(prompt)
  return response

response = generate_response(context, query)
print("Generated Response:\n", response)

In [ ]:
evaluation_dataset = {
    "What is the Transformer architecture?":[1,2,3,4,5],
    "Describe the attention mechanism.":[1,2,3,4,5],
    "Compare the Transformer to traditional RNNs.":[1,2,3,4,5],
    "What are the key contributions of the paper?":[1,2,3,4,5],
    "Explain the concept of self-attention.":[1,2,3,4,5],
    "How does the Transformer handle long-range dependencies?":[1,2,3,4,5],
    "What are the benefits of using multi-head attention?":[1,2,3,4,5],
    "Summarize the experiments and results.":[1,2,3,4,5],
    "What are the main limitations of the Transformer?":[1,2,3,4,5],
    "How does the Transformer perform compared to other models?":[1,2,3,4,5]
}

for query in evaluation_dataset:
  print(query)
  relevant_chunk = relevant_chunks(query, k=5)
  context = " ".join(relevant_chunk)
  print("Context for response:\n", context)
  response = generate_response(context, query)
  print("Generated Response:\n", response)

In [ ]:
def relevant_indices(query, k=5):
  query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().numpy()
  distances, indices = index.search(query_embedding.reshape(1, -1), k)
  relevant_chunks = indices[0]
  return relevant_chunks

def precision_at_k(relevant_docs, retrieved_docs, k):
  retrieved_k = retrieved_docs[:k]
  relevant_retrieved = [doc for doc in retrieved_k if doc in relevant_docs]
  return len(relevant_retrieved) / k

def recall_at_k(relevant_docs, retrieved_docs, k):
  retrieved_k = retrieved_docs[:k]
  relevant_retrieved = [doc for doc in retrieved_k if doc in relevant_docs]
  return len(relevant_retrieved) / len(relevant_docs)

for query in evaluation_dataset:
  relevant_docs = evaluation_dataset[query]
  retrieved_docs = relevant_indices(query, k=5)
  k = 5

  precision = precision_at_k(relevant_docs, retrieved_docs, k)
  recall = recall_at_k(relevant_docs, retrieved_docs, k)
  print(f"Query: {query}")
  print(f"Precision@{k}: {precision}")
  print(f"Recall@{k}: {recall}")
